In [38]:
import praw
from psaw import PushshiftAPI
import json

reddit = praw.Reddit(client_id='D52pMTitN_4Okg',
                     client_secret='6sOjBsPe-EryAiLdvgoRLyFKrVA',
                     user_agent='ChomskyBot',
                     username='ChomskyBot',
                     password='CaRaGio12625')

In [39]:
subreddit = reddit.subreddit('amitheasshole')
api = PushshiftAPI(reddit)

In [40]:
import nltk
from nltk.tokenize import word_tokenize
def label(p):
    labels = ['YTA', 'NTA', 'ESH', 'NAH']
    label_score={label:0 for label in labels}
    
    #If the post hasn't been "flaired" (assigned a label by mods) then I calculate it myself
    if p['link_flair_text'] == None:
        p['label'] = comment_label(p, labels, label_score)
        
    #If the post has been flaired with one of the labels I assign it a 100% probability, otherwise
    #for posts with tags like META I give the post no label
    else:
        if p['link_flair_text'] == 'Asshole':
            label_score['YTA'] = 1
        elif p['link_flair_text'] == 'Not the A-hole':
            label_score['NTA'] = 1
        elif p['link_flair_text'] == 'Everyone Sucks':
            label_score['ESH'] = 1
        elif p['link_flair_text'] == 'No A-Holes here':
            label_score['NAH'] = 1
        else:
            label_score = None
    
    p['label'] = label_score

    
def comment_label(p, labels, label_score):
    
    score = 0
    label = False
    
    #I loop through the comments and try to look for one of the keywords, if I find it I add its score to the
    #tags score
    for c in p['comments']:
        words = word_tokenize(c['body'])
        for w in words:
            if w in labels:
                label = True
                label_score[w] += c['score']
                score += c['score']
    
    for k, v in label_score.items():
        if score != 0:
            label_score[k] = v/score
            
    
    #If no comment contained any of the keywords for a label then I give the post no label
    if not label:
        label_score = None
        
    return label_score
            


In [97]:
import datetime as dt
def post_retrieval(file, subreddit, name, date):

    if name != None:
        submissions = api.search_submissions(after_id=name, subreddit=subreddit, limit=None, sort = "date:asc")
        
    else:
        submissions = api.search_submissions(after=date, subreddit=subreddit, limit=None, sort="date:asc")
        

    for submission in submissions:
        #Retrieve set variables from the submission object and store them in a post dictionary
        post = vars(submission)
        post_dict = {field:post[field] for field in post_fields[:-2]}
        
        #Retrieve 10 top comments and stors them as a dict, this is used for labelling of non-flaired posts
        comments = []
        if post_dict['link_flair_text'] == None:
            for i in range(1, min(11, len(submission.comments.list()))):
                comment = submission.comments.list()[i]
                content = vars(comment)
                comment_dict = {field:content[field] for field in comment_fields if field in content}
                if 'body' in content:
                    comments.append(comment_dict)
        
        post_dict['comments'] = comments
        label(post_dict)
        posts.append(post_dict)
        
        #After 50 posts are added they're written into the data file, this is done to ensure that the progress
        #is not lost in case of internet connection issues or other problems
        if len(posts)%50 == 0:
            print("We're at: " + str(len(posts)))
            with open(file, 'w+') as f:
                json.dump(posts, f)
                
        
                
    date = posts[-1]['created_utc']
    name = posts[-1]['id']

In [98]:

posts = []
post_dict = {}

#Fields to be retrieved from submissions and comments and subreddit from which we will be extracting them
comment_fields = ('id', 'body', 'link_id', 'permalink', 'score', 'subreddit_id')
post_fields = ('id','name', 'created_utc', 'title', 'link_flair_text', 'selftext', 'score', 'comments', 'label')
subreddit = 'amitheasshole'

#Period from which we want to retrieve the comments
date = int(dt.datetime(2019,1,1).timestamp())
now = int(dt.datetime(2019,5,6).timestamp())
name = None

post_retrieval("data.json", name, date)
r = 1
while date < now:
    post_retrieval("data"+str(r)+".json", subreddit, name, date)
    print(date)
    print(name)
    
    r+=1

    

We're at: 50
We're at: 100
We're at: 150
We're at: 200
We're at: 250
We're at: 300
We're at: 350
We're at: 400
We're at: 450
We're at: 500
We're at: 550
We're at: 600
We're at: 650
We're at: 700
We're at: 750
We're at: 800
We're at: 850
We're at: 900
We're at: 950
We're at: 1000
We're at: 1050
We're at: 1100
We're at: 1150
We're at: 1200
We're at: 1250
We're at: 1300
We're at: 1350
We're at: 1400
We're at: 1450
We're at: 1500
We're at: 1550
We're at: 1600
We're at: 1650
We're at: 1700
We're at: 1750
We're at: 1800
We're at: 1850
We're at: 1900
We're at: 1950
We're at: 2000
We're at: 2050
We're at: 2100
We're at: 2150
We're at: 2200
We're at: 2250
We're at: 2300
We're at: 2350
We're at: 2400
We're at: 2450
We're at: 2500
We're at: 2550
We're at: 2600
We're at: 2650
We're at: 2700
We're at: 2750
We're at: 2800
We're at: 2850
We're at: 2900
We're at: 2950
We're at: 3000
We're at: 3050
We're at: 3100
We're at: 3150
We're at: 3200
We're at: 3250
We're at: 3300
We're at: 3350
We're at: 3400


KeyboardInterrupt: 

324


*Whatever*

In [ ]:
with open('data.json', 'a+') as f:
    json.dump(posts, f)

In [15]:

with open('data35.json') as json_file:  
    
    data_r = json.load(json_file)
    print(len(data_r))

ids= [w['id'] for w in data_r]

print(len(ids))
print(len(set(ids)))

    
    

33666
33666
962
